The following post introduces the `petpy` package and its methods for interacting with the Petfinder API. The goal of the `petpy` library is to enable other users to interact with the rich data available in the Petfinder database with an easy-to-use and straightforward Python interface. Methods for coercing the often messy JSON and XML API outputs into [pandas DataFrame](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html) are also available to help facilitate users who are more interested in using the API for data analysis. More information on the Petfinder API itself can be found on the [API documentation page](https://www.petfinder.com/developers/api-docs#methods).

# Table of Contents

* [Installing petpy and Obtaining an API key](#installation)
* [Examples using the `petpy` package](#examples)
    - [Pet Methods](#petmethods)
        - [Extracting breeds of animals available in the Petfinder database](#breeds.list)
        - [Returning random Petfinder pet records](#pet.getRandom)
        - [Return a pet record associated with a specific petId](#pet.get)
        - [Finding pet records matching particular search criteria](#pet.find)
    - [Shelter Methods](#sheltermethods)
        - [Finding shelters matching search criteria](#shelter.find)
        - [Returning specific shelter information](#shelter.get)
        - [Extracting pet records from a particular shelter](#shelter.getPets)

<a id='installation'></a>

## Installation

If not already installed, install `petpy` using `pip`:

``pip install petpy``

Then, import the package.

In [1]:
import petpy

The Petfinder API requires an API key to authenticate access. To receive an API key, register with Petfinder on their developer page: https://www.petfinder.com/developers/api-key

The API key received from Petfinder is then used to authenticate the `Petfinder` class in `petpy`.

The API key is first stored as an environment variable and then loaded using the `os` library. Storing your keys received from APIs and other sensitive information in a secure file or as an environment variable is considered best practice to avoid any potential malicious activity.

In [2]:
import os

key = os.getenv('PETFINDER_KEY')

In [3]:
pf = petpy.Petfinder(key)

The `pf` variable is the initialized Petfinder class with our given API key. We can now use this instance to interact with and extract data from the Petfinder API.

<a id='#examples'></a>

## Examples using the `petpy` package

<a id='petmethods'></a>

## Pet Methods

The following examples demonstrate some simple usage of using `petpy` to interact with and pull data from the Petfinder database. `petpy` contains methods for coercing the returned API results into a pandas DataFrame for easier data analysis and exporting the results into more common formats such as .csv or Excel. More examples of how to use `petpy` in conjunction with the Python scientific computing stack (Scipy, Numpy, pandas, scikit-learn, etc.) to analyze the results can be found in the later chapters of this introduction.

<a id='breed.list'></a>

### Getting Animal Breeds

Pulling the list of animal breeds from the Petfinder database is straightforward with `petpy`. Let's say we are interested in finding the available breeds of cats:

In [4]:
cats = pf.breed_list('cat')

The default return format is JSON, but can be changed to XML by setting the default parameter `outputformat` to 'xml'.

In [5]:
cats

{'@encoding': 'iso-8859-1',
 '@version': '1.0',
 'petfinder': {'@xmlns:xsi': 'http://www.w3.org/2001/XMLSchema-instance',
  '@xsi:noNamespaceSchemaLocation': 'http://api.petfinder.com/schemas/0.9/petfinder.xsd',
  'breeds': {'@animal': 'cat',
   'breed': [{'$t': 'Abyssinian'},
    {'$t': 'American Curl'},
    {'$t': 'American Shorthair'},
    {'$t': 'American Wirehair'},
    {'$t': 'Applehead Siamese'},
    {'$t': 'Balinese'},
    {'$t': 'Bengal'},
    {'$t': 'Birman'},
    {'$t': 'Bobtail'},
    {'$t': 'Bombay'},
    {'$t': 'British Shorthair'},
    {'$t': 'Burmese'},
    {'$t': 'Burmilla'},
    {'$t': 'Calico'},
    {'$t': 'Canadian Hairless'},
    {'$t': 'Chartreux'},
    {'$t': 'Chausie'},
    {'$t': 'Chinchilla'},
    {'$t': 'Cornish Rex'},
    {'$t': 'Cymric'},
    {'$t': 'Devon Rex'},
    {'$t': 'Dilute Calico'},
    {'$t': 'Dilute Tortoiseshell'},
    {'$t': 'Domestic Long Hair'},
    {'$t': 'Domestic Medium Hair'},
    {'$t': 'Domestic Short Hair'},
    {'$t': 'Egyptian Mau'},

The `return_df` parameter can also be set to True to coerce the results into a pandas DataFrame.

In [6]:
cats_df = pf.breed_list('cat', return_df=True)
cats_df.head()

,cat breeds
0,Abyssinian
1,American Curl
2,American Shorthair
3,American Wirehair
4,Applehead Siamese


Please note the coercion to a pandas DataFrame removes the metadata returned in the JSON format to make the conversion process more efficient and straightforward.

According to Petfinder's API documentation, the available animals to search are ['barnyard', 'bird', 'cat', 'dog', 'horse', 'reptile', 'smallfurry']. Searching for an animal not available in the Petfinder database will return a JSON object with a message stating 'invalid arguments'.

In [7]:
pf.breed_list('zebra')

{'@encoding': 'iso-8859-1',
 '@version': '1.0',
 'petfinder': {'@xmlns:xsi': 'http://www.w3.org/2001/XMLSchema-instance',
  '@xsi:noNamespaceSchemaLocation': 'http://api.petfinder.com/schemas/0.9/petfinder.xsd',
  'header': {'status': {'code': {'$t': '200'},
    'message': {'$t': 'invalid arguments'}},
   'timestamp': {'$t': '2018-03-15T22:37:27Z'},
   'version': {'$t': '0.1'}}}}

<a id='pet.getRandom'></a>

### Returning random Petfinder pet records

The `petpy` method `pet_getRandom()` provides a wrapper for the Petfinder `pet.getRandom` method. The potential results can be filtered to a subset by the method parameters, otherwise the method can be called simply as:

In [8]:
pf.pet_get_random()

{'@encoding': 'iso-8859-1',
 '@version': '1.0',
 'petfinder': {'@xmlns:xsi': 'http://www.w3.org/2001/XMLSchema-instance',
  '@xsi:noNamespaceSchemaLocation': 'http://api.petfinder.com/schemas/0.9/petfinder.xsd',
  'header': {'status': {'code': {'$t': '100'}, 'message': {}},
   'timestamp': {'$t': '2018-03-15T22:37:33Z'},
   'version': {'$t': '0.1'}},
  'petIds': {'id': {'$t': '41123651'}}}}

The default record output contains only the pet record's ID and the call's JSON metadata. If we wish to return a more complete random pet record, we can set the parameter `output` to `basic` (name, age, animal, breed, shelterID) or `full` (complete record with description).

In [9]:
pf.pet_get_random(output='full')

{'@encoding': 'iso-8859-1',
 '@version': '1.0',
 'petfinder': {'@xmlns:xsi': 'http://www.w3.org/2001/XMLSchema-instance',
  '@xsi:noNamespaceSchemaLocation': 'http://api.petfinder.com/schemas/0.9/petfinder.xsd',
  'header': {'status': {'code': {'$t': '100'}, 'message': {}},
   'timestamp': {'$t': '2018-03-15T22:37:35Z'},
   'version': {'$t': '0.1'}},
  'pet': {'age': {'$t': 'Young'},
   'animal': {'$t': 'Dog'},
   'breeds': {'breed': {'$t': 'Rat Terrier'}},
   'contact': {'address1': {},
    'address2': {},
    'city': {'$t': 'Springtown'},
    'email': {'$t': 'lcrescue@aol.com'},
    'fax': {},
    'phone': {'$t': '682-229-9064 /  817-823-0750'},
    'state': {'$t': 'TX'},
    'zip': {'$t': '76082'}},
   'description': {'$t': "My name is Tobias and I'm a sweet little boy I love to cuddle I like sleeping in my doggie bed but I would prefer to be in my doggie bed right next to your bed I'm least trained crate trained good with kids and other dogs and just really need a home of my own"},

We can also pull a specified number of pet records from the API by setting the `records` parameter and return the collected results as a pandas DataFrame by setting `return_df` to `True`.

In [10]:
random_pet_df = pf.pet_get_random(records=5, return_df=True)

<a id='pet.get'></a>

### Return a pet record associated with a specific petId

The `pet_get` method can be used to extract a full record from the Petfinder database. We use the pet ID retrieved from the previous call to `pet_getRandom` to illustrate.

In [12]:
pet = pf.pet_get(random_pet_df['id'][:1].values[0])

In [13]:
pet

{'@encoding': 'iso-8859-1',
 '@version': '1.0',
 'petfinder': {'@xmlns:xsi': 'http://www.w3.org/2001/XMLSchema-instance',
  '@xsi:noNamespaceSchemaLocation': 'http://api.petfinder.com/schemas/0.9/petfinder.xsd',
  'header': {'status': {'code': {'$t': '100'}, 'message': {}},
   'timestamp': {'$t': '2018-03-15T22:37:47Z'},
   'version': {'$t': '0.1'}},
  'pet': {'age': {'$t': 'Young'},
   'animal': {'$t': 'Cat'},
   'breeds': {'breed': {'$t': 'Tabby'}},
   'contact': {'address1': {'$t': '374 Worthington Mill Road'},
    'address2': {},
    'city': {'$t': 'Richboro'},
    'email': {'$t': 'linsatter@verizon.net'},
    'fax': {},
    'phone': {'$t': '215-801-6398 '},
    'state': {'$t': 'PA'},
    'zip': {'$t': '18954'}},
   'description': {'$t': 'Be is very shy and loves other cats.\n\nDetails:  Female, Tabby, 1 year old, Spayed'},
   'id': {'$t': '37616397'},
   'lastUpdate': {'$t': '2017-03-12T23:08:48Z'},
   'media': {'photos': {'photo': [{'$t': 'http://photos.petfinder.com/photos/pets/

The record can also be returned as a `DataFrame`.

In [14]:
pf.pet_get(random_pet_df['id'][:1].values[0], return_df=True)

,age,animal,breeds.breed,address1,city,email,phone,state,zip,description,...,size,status,breed0,status0,status1,photos0,photos1,photos2,photos3,photos4
0,Young,Cat,Tabby,374 Worthington Mill Road,Richboro,linsatter@verizon.net,215-801-6398,PA,18954,Be is very shy and loves other cats.\n\nDetail...,...,M,A,Tabby,altered,hasShots,http://photos.petfinder.com/photos/pets/376163...,http://photos.petfinder.com/photos/pets/376163...,http://photos.petfinder.com/photos/pets/376163...,http://photos.petfinder.com/photos/pets/376163...,http://photos.petfinder.com/photos/pets/376163...


The `pets_get()` method accepts a list or tuple and returns the records associated with each pet ID in the passed variable.

In [15]:
petids = random_pet_df['id'].tolist() # get the pet IDs from the previous call by turning the id column into a list

In [16]:
petids

['37616397', '39874883', '40598343', '40565836', '40514031']

In [17]:
pf.pets_get(petids, return_df=True)

,address1,address2,age,animal,breed0,breeds.breed,city,description,email,id,...,photos9,sex,shelterId,shelterPetId,size,state,status,status0,status1,zip
0,374 Worthington Mill Road,NaN,Young,Cat,Tabby,Tabby,Richboro,Be is very shy and loves other cats.\n\nDetail...,linsatter@verizon.net,37616397,...,NaN,F,PA524,NaN,M,PA,A,altered,hasShots,18954
0,NaN,NaN,Adult,Dog,Miniature Pinscher,Miniature Pinscher,Baton Rouge,ADOPTION PENDING--Trooper is male Miniature Pi...,rescue.me.mail@gmail.com,39874883,...,http://photos.petfinder.com/photos/pets/398748...,M,LA279,NaN,S,LA,A,altered,hasShots,70814
0,12801 S. IH 35,PO BOX 394,Baby,Dog,Chihuahua,Chihuahua,Jarrell,HI! ISN'T MY NAME KIND OF FUNNY? THEY CALL ME ...,midnightsjourney@aol.com,40598343,...,http://photos.petfinder.com/photos/pets/405983...,F,TX1875,11262965,S,TX,A,specialNeeds,altered,76537
0,5345 Hwy 12,NaN,Senior,Dog,Pit Bull Terrier,Pit Bull Terrier,Santa Rosa,Molly (the blue and white girl) and Travis (th...,hsscadoptions@humanesocietysoco.org,40565836,...,http://photos.petfinder.com/photos/pets/405658...,F,CA697,17647,L,CA,A,NaN,NaN,95401
0,2000 N. Roosevelt Ave.,NaN,Adult,Dog,Hound,Hound,Burlington,"****Please call the shelter with questions, 31...",dmchumane@yahoo.com,40514031,...,NaN,M,IA04,NaN,L,IA,A,NaN,NaN,52601


The `pets_get()` method is essentially a convenience wrapper of `pet_get()`. The same results can be obtained by passing the variable to `pet_get()`.

In [18]:
pf.pet_get(petids, return_df=True)

,address1,address2,age,animal,breed0,breeds.breed,city,description,email,id,...,photos9,sex,shelterId,shelterPetId,size,state,status,status0,status1,zip
0,374 Worthington Mill Road,NaN,Young,Cat,Tabby,Tabby,Richboro,Be is very shy and loves other cats.\n\nDetail...,linsatter@verizon.net,37616397,...,NaN,F,PA524,NaN,M,PA,A,altered,hasShots,18954
0,NaN,NaN,Adult,Dog,Miniature Pinscher,Miniature Pinscher,Baton Rouge,ADOPTION PENDING--Trooper is male Miniature Pi...,rescue.me.mail@gmail.com,39874883,...,http://photos.petfinder.com/photos/pets/398748...,M,LA279,NaN,S,LA,A,altered,hasShots,70814
0,12801 S. IH 35,PO BOX 394,Baby,Dog,Chihuahua,Chihuahua,Jarrell,HI! ISN'T MY NAME KIND OF FUNNY? THEY CALL ME ...,midnightsjourney@aol.com,40598343,...,http://photos.petfinder.com/photos/pets/405983...,F,TX1875,11262965,S,TX,A,specialNeeds,altered,76537
0,5345 Hwy 12,NaN,Senior,Dog,Pit Bull Terrier,Pit Bull Terrier,Santa Rosa,Molly (the blue and white girl) and Travis (th...,hsscadoptions@humanesocietysoco.org,40565836,...,http://photos.petfinder.com/photos/pets/405658...,F,CA697,17647,L,CA,A,NaN,NaN,95401
0,2000 N. Roosevelt Ave.,NaN,Adult,Dog,Hound,Hound,Burlington,"****Please call the shelter with questions, 31...",dmchumane@yahoo.com,40514031,...,NaN,M,IA04,NaN,L,IA,A,NaN,NaN,52601


<a id='pet.find'></a>

### Finding pet records matching particular search criteria

The `pet.find()` method returns a collection of pet records that match the input search criteria. The available search criteria are listed in the [petpy API documentation](http://petpy.readthedocs.io/en/latest/api.html#pet-methods).

For example, let's say we are interested in finding female cats in Washington state and we want the results returned in a tidy pandas DataFrame.

In [19]:
cats = pf.pet_find(location='WA', animal='cat', sex='F', return_df=True)

In [20]:
cats.head()

,age,animal,address1,city,email,phone,state,zip,description,id,...,photos20,photos21,photos22,photos23,photos24,photos25,photos26,photos27,photos28,photos29
0,Senior,Cat,21615 64th Ave S,Kent,adoptapet@kingcounty.gov,206-296-7387,WA,98032,Boo Kitty / Blue Kitty (is that a Russian Blue...,39921493,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,Adult,Cat,21615 64th Ave S,Kent,adoptapet@kingcounty.gov,206-296-7387,WA,98032,This is my friend Gertie. She is an 11-year-ol...,37949653,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,Senior,Cat,8103 161st Ave NE,Redmond,info@thewhole-cat.com,425-576-5548,WA,98052,This cat is currently available for adoption a...,40688475,...,http://photos.petfinder.com/photos/pets/406884...,http://photos.petfinder.com/photos/pets/406884...,http://photos.petfinder.com/photos/pets/406884...,http://photos.petfinder.com/photos/pets/406884...,http://photos.petfinder.com/photos/pets/406884...,NaN,NaN,NaN,NaN,NaN
0,Senior,Cat,8103 161st Ave NE,Redmond,info@thewhole-cat.com,425-576-5548,WA,98052,This cat is currently available for adoption a...,40941038,...,http://photos.petfinder.com/photos/pets/409410...,http://photos.petfinder.com/photos/pets/409410...,http://photos.petfinder.com/photos/pets/409410...,http://photos.petfinder.com/photos/pets/409410...,http://photos.petfinder.com/photos/pets/409410...,NaN,NaN,NaN,NaN,NaN
0,Senior,Cat,8103 161st Ave NE,Redmond,info@thewhole-cat.com,425-576-5548,WA,98052,This cat is currently available for adoption a...,40998129,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


The default amount of records returned is 25, which can be changed by setting the `count` parameter. For large queries, it is recommended to set the `pages` parameter with a smaller `count` value. For example, if we wanted to return 1000 results, we could set the `pages` parameter to 10 and the `count` parameter to 100. Please note the Petfinder API places a hard cap of 2,000 results returned per query.

<a id='sheltermethods'></a>

## Shelter Methods

Shelter methods are quite similar to the pet methods explored previously but return information on the animal welfare organizations available in Petfinder's database.

<a id='shelter.find'></a>

### Finding animal welfare organizations in a certain area

The `shelter_find()` method can be used to return shelter records matching the input search criteria. Let's say we want to find 10 shelters listed in the Petfinder database located in Washington State as a pandas DataFrame.

In [21]:
wa_shelters = pf.shelter_find(location='WA', count=10, return_df=True)

In [22]:
wa_shelters

,address1,address2,city,country,email,id,latitude,longitude,name,phone,state,zip
0,NaN,NaN,Auburn,US,pugetsoundrescue@hotmail.com,WA149,47.3191,-122.2682,Puget Sound Rescue,NaN,WA,98092
1,NaN,NaN,Auburn,US,foggycreekcavyrescue@yahoo.com,WA254,47.3191,-122.2682,Foggy Creek Cavy Rescue,NaN,WA,98092
2,21615 64th Ave S,NaN,Kent,US,adoptapet@kingcounty.gov,WA252,47.3747,-122.2775,Regional Animal Services of King County - Fost...,206-296-7387,WA,98032
3,21615 64th Ave S.,NaN,Kent,US,adoptapet@kingcounty.gov,WA63,47.3747,-122.2775,Regional Animal Services of King County,(206) 296-7387,WA,98032
4,NaN,NaN,Redmond,US,pacosrescue@yahoo.com,WA604,47.3311,-122.3592,Pacos Rescue Network,NaN,WA,98052
5,NaN,NaN,Federal Way,US,rescuinganimals@gmail.com,WA660,47.3135,-122.3627,RAIN - Rescuing Animals In Need,NaN,WA,98023
6,PO Box 13244,NaN,Des Moines,US,anotherchancecats2007@gmail.com,WA401,47.4043,-122.3105,Another Chance Cat Adoption,(253) 856-1771,WA,98198
7,16915 SE 272nd St.,#100-210,Covington,US,smidgetrescue@hotmail.com,WA544,47.3718,-122.1474,Smidget Rescue,206.817.3731,WA,98042
8,NaN,NaN,Covington,US,adoptions.leftbehindk9@gmail.com,WA555,47.3718,-122.1474,Left Behind K-9 Rescue,NaN,WA,98042
9,1102 E. Main Ave,NaN,Puyallup,US,info@sunnyskysshelter.org,WA470,47.2032,-122.2738,Sunny Sky's Animal Rescue,NaN,WA,98372


<a id='shelter.get'></a>

### Returning specific shelter information

The `shelter_get()` method returns the available information in the Petfinder database matching the given shelter ID. Shelter IDs can be found using the `shelter_find()` method used earlier. For example, let's use the method to return the record matching the first shelter ID in the result set obtained in the previous example.

In [23]:
shelter_list = wa_shelters['id'].tolist()

In [24]:
pf.shelter_get(shelter_list[0])

{'@encoding': 'iso-8859-1',
 '@version': '1.0',
 'petfinder': {'@xmlns:xsi': 'http://www.w3.org/2001/XMLSchema-instance',
  '@xsi:noNamespaceSchemaLocation': 'http://api.petfinder.com/schemas/0.9/petfinder.xsd',
  'header': {'status': {'code': {'$t': '100'}, 'message': {}},
   'timestamp': {'$t': '2018-03-15T22:38:04Z'},
   'version': {'$t': '0.1'}},
  'shelter': {'address1': {},
   'address2': {},
   'city': {'$t': 'Auburn'},
   'country': {'$t': 'US'},
   'email': {'$t': 'pugetsoundrescue@hotmail.com'},
   'fax': {},
   'id': {'$t': 'WA149'},
   'latitude': {'$t': '47.3191'},
   'longitude': {'$t': '-122.2682'},
   'name': {'$t': 'Puget Sound Rescue'},
   'phone': {},
   'state': {'$t': 'WA'},
   'zip': {'$t': '98092'}}}}

The `shelter_get()` method can also accept a list or tuple of shelter IDs. Internally, this calls a convenience wrapper method `shelters_get()`.

In [25]:
pf.shelters_get(shelter_list, return_df=True)

,address1,address2,city,country,email,id,latitude,longitude,name,phone,state,zip
0,NaN,NaN,Auburn,US,pugetsoundrescue@hotmail.com,WA149,47.3191,-122.2682,Puget Sound Rescue,NaN,WA,98092
0,NaN,NaN,Auburn,US,foggycreekcavyrescue@yahoo.com,WA254,47.3191,-122.2682,Foggy Creek Cavy Rescue,NaN,WA,98092
0,21615 64th Ave S,NaN,Kent,US,adoptapet@kingcounty.gov,WA252,47.3747,-122.2775,Regional Animal Services of King County - Fost...,206-296-7387,WA,98032
0,21615 64th Ave S.,NaN,Kent,US,adoptapet@kingcounty.gov,WA63,47.3747,-122.2775,Regional Animal Services of King County,(206) 296-7387,WA,98032
0,NaN,NaN,Redmond,US,pacosrescue@yahoo.com,WA604,47.3311,-122.3592,Pacos Rescue Network,NaN,WA,98052
0,NaN,NaN,Federal Way,US,rescuinganimals@gmail.com,WA660,47.3135,-122.3627,RAIN - Rescuing Animals In Need,NaN,WA,98023
0,PO Box 13244,NaN,Des Moines,US,anotherchancecats2007@gmail.com,WA401,47.4043,-122.3105,Another Chance Cat Adoption,(253) 856-1771,WA,98198
0,16915 SE 272nd St.,#100-210,Covington,US,smidgetrescue@hotmail.com,WA544,47.3718,-122.1474,Smidget Rescue,206.817.3731,WA,98042
0,NaN,NaN,Covington,US,adoptions.leftbehindk9@gmail.com,WA555,47.3718,-122.1474,Left Behind K-9 Rescue,NaN,WA,98042
0,1102 E. Main Ave,NaN,Puyallup,US,info@sunnyskysshelter.org,WA470,47.2032,-122.2738,Sunny Sky's Animal Rescue,NaN,WA,98372


The result obtained would be the same if one were to use the `shelter_get()` method and passed the same variable.

<a id='shelter.getPets'></a>

### Extracting pet records from a particular shelter

The `shelter.getPets()` method returns the pet records that belong to a particular shelter ID. For example, let's say we want to return the pet records from the first shelter in our list as a DataFrame.

In [26]:
pf.shelter_get_pets(shelter_list[0], return_df=True)

,age,animal,city,email,state,zip,description,id,lastUpdate,media.photos.photo,...,photos20,photos21,photos22,photos23,photos24,photos25,photos26,photos27,photos28,photos29
0,Adult,Dog,Auburn,pugetsoundrescue@hotmail.com,WA,98092,"Morena is a shepherd/lab mix we are guessing, ...",32080691,2015-05-05T13:34:46Z,"[{'@size': 'pnt', '$t': 'http://photos.petfind...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,Adult,Dog,Auburn,pugetsoundrescue@hotmail.com,WA,98092,Monty is a loving gentleman that likes long wa...,34906970,2016-04-15T19:21:14Z,"[{'@size': 'pnt', '$t': 'http://photos.petfind...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,Adult,Dog,Auburn,pugetsoundrescue@hotmail.com,WA,98092,ADOPTION DONATION: $350\n\nCONTACT US TODAY: P...,37760609,2017-04-02T01:32:08Z,"[{'@size': 'pnt', '$t': 'http://photos.petfind...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,Young,Dog,Auburn,pugetsoundrescue@hotmail.com,WA,98092,ADOPTION DONATION: $350\n\nCONTACT US TODAY: P...,38602092,2017-06-28T05:40:59Z,"[{'@size': 'pnt', '$t': 'http://photos.petfind...",...,http://photos.petfinder.com/photos/pets/386020...,http://photos.petfinder.com/photos/pets/386020...,http://photos.petfinder.com/photos/pets/386020...,http://photos.petfinder.com/photos/pets/386020...,http://photos.petfinder.com/photos/pets/386020...,NaN,NaN,NaN,NaN,NaN
0,Adult,Dog,Auburn,pugetsoundrescue@hotmail.com,WA,98092,ADOPTION DONATION: $350\n\nCONTACT US TODAY: P...,38900079,2017-07-27T07:02:18Z,"[{'@size': 'pnt', '$t': 'http://photos.petfind...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,Adult,Dog,Auburn,pugetsoundrescue@hotmail.com,WA,98092,ADOPTION DONATION: $350\n\nCONTACT US TODAY: P...,39075204,2017-08-12T21:12:40Z,"[{'@size': 'pnt', '$t': 'http://photos.petfind...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,Senior,Dog,Auburn,pugetsoundrescue@hotmail.com,WA,98092,ADOPTION DONATION: $350\n\nCONTACT US TODAY: P...,39229895,2017-08-27T05:37:10Z,"[{'@size': 'pnt', '$t': 'http://photos.petfind...",...,http://photos.petfinder.com/photos/pets/392298...,http://photos.petfinder.com/photos/pets/392298...,http://photos.petfinder.com/photos/pets/392298...,http://photos.petfinder.com/photos/pets/392298...,http://photos.petfinder.com/photos/pets/392298...,http://photos.petfinder.com/photos/pets/392298...,http://photos.petfinder.com/photos/pets/392298...,http://photos.petfinder.com/photos/pets/392298...,http://photos.petfinder.com/photos/pets/392298...,http://photos.petfinder.com/photos/pets/392298...
0,Senior,Dog,Auburn,pugetsoundrescue@hotmail.com,WA,98092,We just took in Grady a few days ago. Grady i...,39793927,2017-10-31T19:38:10Z,"[{'@size': 'pnt', '$t': 'http://photos.petfind...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,Young,Dog,Auburn,pugetsoundrescue@hotmail.com,WA,98092,ADOPTION DONATION: $350\n\nCONTACT US TODAY: P...,39899643,2017-11-13T02:51:58Z,"[{'@size': 'pnt', '$t': 'http://photos.petfind...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,Senior,Dog,Auburn,pugetsoundrescue@hotmail.com,WA,98092,"Eager to please, affectionate, quiet, good on ...",39899697,2017-11-13T02:58:37Z,"[{'@size': 'pnt', '$t': 'http://photos.petfind...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
